Before starting, make sure you have plotly and colorlover and pycountry in python by running the following on terminal: 

sudo pip install plotly 

sudo pip install colorlover

sudo pip install pycountry

In [1]:
import csv

#returns the csv in the form of a list of lists
def csvListMaker(nameOfFile):
    listOfAll = []
    with open(nameOfFile, 'rb') as streetfoodfile:
        csvreader = csv.reader(streetfoodfile)
        for row in csvreader:
            #each row is a list where index 0 is food and index 1 is the location
            listOfAll.append(row)
    return listOfAll

#returns list of only the 0th index which should be the foods
def foodListMaker(nameOfFile):
    listOfFoods = []
    with open(nameOfFile, 'rb') as streetfoodfile:
        csvreader = csv.reader(streetfoodfile)
        for row in csvreader:
            #each row is a list where index 0 is food and index 1 is the location
            listOfFoods.append(row[0])
    return listOfFoods


In [2]:
from pandas import *

international_list = csvListMaker("InternationalFoods.csv")[1:]

international_dataframe= DataFrame(international_list) #printing as data frame just because then it's easy to see which index goes wtih what


In [3]:
international_dataframe

,0,1,2,3,4,5
0,Biscuit,USA,International,,,
1,Apple butter,USA,International,,,
2,Apple sauce,USA,International,,,
3,Baked potato,USA,International,,,
4,Barbecue,USA,International,,,
5,Bear claw,USA,International,,,
6,Beef Manhattan,USA,International,,,
7,Blue cheese dressing,USA,International,,,
8,Brunswick stew,USA,International,,,
9,Buffalo burger,USA,International,,,


In [4]:
#get the list of countries
no_country_duplicates = international_dataframe.drop_duplicates(subset= [1], keep='first', inplace=False)
countries_list = no_country_duplicates[1].tolist()
print countries_list

['USA', 'China', 'France', 'Germany', 'UK', 'India', 'Italy', 'Thailand', 'Spain', 'Mexico', 'Ethiopia', 'Ecuador', 'Korea', 'Vietnam', 'Cameroon', 'Nigeria', 'Morocco', 'North Africa', 'Levant', 'South Africa', 'Libya', 'Southern Africa', 'Tunisia', 'Zimbabwe', 'Egypt', 'Ghana', 'Algeria', 'West Africa', 'East Africa', 'Uganda', 'Maghreb', 'Central Africa', 'Namibia', 'Botswana', 'Northwest Africa', 'Senegal', 'African Great Lakes', 'Africa']


In [5]:
#setup pytrends
from pytrends.request import TrendReq

pytrends = TrendReq(hl='en-US', tz=360)
                    # ,proxies = {'https': 'https://34.203.233.13:80'})


In [6]:
import pandas as pd

#get the Google trends over time for items in dataframe df
def info_over_time (df, start_date, end_date, region, first_item, country=None):
    
    time = start_date + ' ' + end_date
    
    #if a country is specified, only get items for that country
    if country == None:
        kwlist = df[0].tolist()
    else:
        country_items = df.loc[df[1] == country]
        kwlist = country_items[0].tolist()
        
    if first_item == 'None':
        first_item = kwlist[0]
 
    first = [first_item]
        
    #get info for first 5 items
    first.extend(kwlist[0:4])
        
    pytrends.build_payload(first, cat=0, timeframe= time, geo=region, gprop='')
        
    tempdf = pytrends.interest_over_time()
    tempdf = tempdf.drop('isPartial', 1) #drop isPartial col
    tempdf.loc['mean'] = tempdf.mean() #add mean as last row
    
    ref_mean = tempdf.loc['mean'][0] #mean of first term, used as reference    
    df = tempdf #final dataframe
    
    # start and end indices of items
    start = 0
    end = 4
    
    done = False
    while not done:
        
        start = start + 4
        if (len(kwlist) - 1) > (end + 4):
            end = end + 4

        else:
            end = len(kwlist) - 1
            done = True
        
        #get the info for these items
        items = kwlist[start:end] 
        
        items.append(first_item)
                
        pytrends.build_payload(items, cat=0, timeframe= time, geo=region, gprop='')
        tempdf = pytrends.interest_over_time()
        tempdf = tempdf.drop('isPartial', 1) #drop isPartial col
        tempdf.loc['mean'] = tempdf.mean() #add mean as last row
        
        #normalize the values
        #equation = x * ref_mean/ ref_mean2
        ref_mean2 = tempdf.loc['mean'][first_item]
                
        normal_factor = float (ref_mean)/ref_mean2
        tempdf = tempdf.apply(lambda x: x*normal_factor)
        tempdf = tempdf.drop(first_item,1)
        
        #append to the final dataframe
        df = pd.concat([df, tempdf], axis=1)

    return df


In [7]:
import pandas as pd

#get the Google trends by region for items in dataframe df
def info_by_region (df, places_list, resolution, start_date, end_date, region, first_item, country=None):
    
    time = start_date + ' ' + end_date
    
    #if a country is specified, only get items for that country
    if country == None:
        kwlist = df[0].tolist()
    else:
        country_items = df.loc[df[1] == country]
        kwlist = country_items[0].tolist()
        
    if first_item == 'None':
        first_item = kwlist[0]
        
    first = [first_item]
        
    #get info for first 5 items
    first.extend(kwlist[0:4])
    pytrends.build_payload(first, cat=0, timeframe= time, geo=region, gprop='')
    tempdf = pytrends.interest_by_region(resolution= resolution)
    tempdf.loc['mean'] = tempdf.mean() #add mean as last row
    ref_mean = tempdf.loc['mean'][0] #mean of first term, used as reference    
    df = tempdf #final dataframe
    
    # start and end indices of items
    start = 0
    end = 4
    
    done = False
    while not done:
        
        start = start + 4
        if (len(kwlist) - 1) > (end + 4):
            end = end + 4

        else:
            end = len(kwlist) 
            done = True
        
        #get the info for these items
        items = kwlist[start:end] 
        items.append(first_item)
        pytrends.build_payload(items, cat=0, timeframe= time, geo=region, gprop='')
        tempdf = pytrends.interest_by_region(resolution= resolution)
        tempdf.loc['mean'] = tempdf.mean() #add mean as last row
        
        #normalize the values
        #equation = x * ref_mean/ ref_mean2
        ref_mean2 = tempdf.loc['mean'][first_item]
        normal_factor = float (ref_mean)/ref_mean2
        tempdf = tempdf.apply(lambda x: x*normal_factor)
        tempdf = tempdf.drop(first_item,1)
        
        #append to the final dataframe
        df = pd.concat([df, tempdf], axis=1)
        
    df = df.loc[places_list, :]

    return df

In [8]:
#get x number of items with the greatest mean
def sortDataFrameAndReturnXRows(unsortedDf, numRowsNeeded):
    sortedDf = unsortedDf.T
    sortedDf = sortedDf.sort_values(by=['mean'], ascending = False)
    sortedDf = sortedDf.iloc[: numRowsNeeded] #select only the top numRowsNeeded rows from the sorted data frame
    return sortedDf.T

In [9]:
from itertools import izip_longest

#helps group and iterate through elements in a list
def grouper(n, iterable, fillvalue=None):
    "grouper(3, 'ABCDEFG', 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return izip_longest(fillvalue=fillvalue, *args)

#list of countries split into more lists to prevent overloading 
#and to help debugging when scraping from Google Trends
split_countries = []
for x, y, z in grouper(3,countries_list):
    split_countries.append([x, y, z])


In [10]:
import pandas as pd

#get top 20 items for each country
def get_top20 (countries, top_list):
    for c in countries:
        print c
        df=info_over_time(international_dataframe, '2013-05-05', '2018-04-22', '', 'None', c)
        condensed_df = sortDataFrameAndReturnXRows(df, 20)
        items = list(condensed_df)
        
        print items
        
        if len(items) >= 20:
            print 'added'
            items_and_country = []
            for i in items:
                items_and_country.append([i, c])
            top_list = top_list.append(pd.DataFrame(items_and_country))
    return top_list


In [12]:
# get top 20 items from each country
top20 = DataFrame()
top20 = get_top20 (countries_list, top20)

USA
['Hamburger', 'Pancakes', 'Barbecue ', 'Biscuit ', 'Fried chicken', 'Meatloaf', 'Baked potato', 'Sour cream', 'Potato salad', 'Coleslaw', 'Chowder', 'Pumpkin pie', 'Pasta salad', 'Corned beef', 'Stuffed peppers', 'Grits', 'Macaroni and cheese', 'Doughnut', 'Chili con carne', 'Chicken sandwich']
added
China
['Fried Rice', 'Dumplings', 'Dim sum', 'Noodle soup', 'Hot pot', 'Chow mein', 'Wonton', 'Peking Duck', 'Kung Pao chicken', 'Sweet and sour pork', 'Braised pork', 'Tea egg', 'Century egg', 'Baozi', 'Soy egg', 'Pickled vegetables', 'Mantou', 'Lamian', 'Jiaozi', 'Cha siu']
added
France
['Ratatouille ', 'Madeline', 'Croissant', 'Baguette', 'Macarons', 'Foie gras', 'Raclette ', 'Chichi ', 'Creme brulee', 'Navette ', 'Mousse au chocolat', 'Gateau au yaourt', 'Tartiflette ', 'Tourte', 'Gratin dauphinois ', 'Tarte Tatin', 'Galette des rois', 'Quenelle ', 'Magret de canard', 'Pain perdu']
added
Germany
['Sauerkraut', 'Marzipan', 'Kartoffelsalat', 'Apfelkuchen', 'Bratwurst', 'Currywurst', 

In [13]:
top20

,0,1
0,Hamburger,USA
1,Pancakes,USA
2,Barbecue,USA
3,Biscuit,USA
4,Fried chicken,USA
5,Meatloaf,USA
6,Baked potato,USA
7,Sour cream,USA
8,Potato salad,USA
9,Coleslaw,USA


In [14]:
no_duplicates = top20.drop_duplicates(subset= [1], keep='first', inplace=False)
final_countries_list = no_duplicates[1].tolist()
print final_countries_list

['USA', 'China', 'France', 'Germany', 'UK', 'India', 'Italy', 'Thailand', 'Spain', 'Mexico', 'Ethiopia', 'Ecuador', 'Korea', 'Vietnam', 'South Africa']


In [15]:
revised_countries_list = ['United States', 'China','France', 'Germany', 'United Kingdom', 'India', 'Italy', 'Thailand', 'Spain', 'Mexico', 'Ethiopia', 'Ecuador', 'South Korea', 'Vietnam', 'South Africa']

In [16]:
#get Google trends over time for top items in each country
top_international_df = info_over_time (top20, '2013-05-05', '2018-04-22', '', 'Chicken sandwich', country=None)


In [17]:
top_international_df

,Chicken sandwich,Hamburger,Pancakes,Barbecue,Biscuit,Fried chicken,Meatloaf,Baked potato,Sour cream,Potato salad,...,Pampoenkoekies,Potbrood,Gatsby Sandwich,Skilpadjies,Braaibroodjies,Pap en Vleis,Hertzoggie,Droëwors,Bar-One Cake,Mielie brood/Mealie bread
date,,,,,,,,,,,,,,,,,,,,,
2013-05-05 00:00:00,3.000000,37.000000,29.000000,39.000000,24.00,20.946733,14.446023,8.667614,7.584162,9.007972,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2013-05-12 00:00:00,3.000000,34.000000,28.000000,30.000000,23.00,19.502131,13.362571,8.306463,7.223011,8.107174,...,0.000000,0.000000,0.057020,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2013-05-19 00:00:00,3.000000,35.000000,27.000000,33.000000,22.00,26.363991,13.362571,7.945312,7.584162,10.809566,...,0.000000,0.000000,0.057020,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2013-05-26 00:00:00,3.000000,36.000000,28.000000,36.000000,21.00,20.946733,13.001420,8.306463,6.861861,14.412755,...,0.000000,0.000000,0.057020,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2013-06-02 00:00:00,4.000000,32.000000,25.000000,43.000000,22.00,19.863281,13.362571,7.584162,6.861861,9.908769,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2013-06-09 00:00:00,3.000000,34.000000,27.000000,40.000000,22.00,20.224432,12.640270,7.945312,7.584162,10.809566,...,0.057020,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2013-06-16 00:00:00,3.000000,33.000000,27.000000,38.000000,21.00,20.585582,11.917969,7.584162,7.584162,10.809566,...,0.057020,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2013-06-23 00:00:00,3.000000,34.000000,29.000000,34.000000,23.00,21.307884,12.279119,7.584162,7.223011,10.809566,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2013-06-30 00:00:00,4.000000,38.000000,30.000000,49.000000,22.00,23.113636,10.834517,8.306463,8.667614,23.420726,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
#get sum of searches in a specified country for each cuisine 
def food_in_country_over_time (df, countries, start_date, end_date, region):
    dataframe = DataFrame()
    for c in countries:
        print c
        tempdf=info_over_time(df, start_date, end_date, region, 'None', c)
        tempdf.loc[:, c] = tempdf.sum(axis=1)
        tempdf = tempdf.loc[:, c] #save sum for country as df
        
        #append to the final dataframe
        dataframe = pd.concat([dataframe, tempdf], axis=1)
    return dataframe

In [19]:
#2 letter codes that correspond to each country
codes = ['US', 'CN', 'FR', 'DE', 'GB', 'IN', 'IT','TH', 'ES','MX', 'ET','EC', 'KR','VN','ZA']
country_and_codes= zip(final_countries_list, codes)
print country_and_codes

[('USA', 'US'), ('China', 'CN'), ('France', 'FR'), ('Germany', 'DE'), ('UK', 'GB'), ('India', 'IN'), ('Italy', 'IT'), ('Thailand', 'TH'), ('Spain', 'ES'), ('Mexico', 'MX'), ('Ethiopia', 'ET'), ('Ecuador', 'EC'), ('Korea', 'KR'), ('Vietnam', 'VN'), ('South Africa', 'ZA')]


In [20]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from pytrends.request import TrendReq
import pandas as pd 

import plotly.graph_objs as go

init_notebook_mode(connected=True)

import plotly
from plotly.graph_objs import Scatter, Layout
import plotly.graph_objs
import plotly.plotly as py
import plotly.figure_factory as ff

In [21]:
#Scatter plot
def draw_scatter(df):
    data = []
    for item in df.iteritems(): 
        #Extract the title: 
        title = item[0]
        #Extract the series of that title 
        s = item[1]
        #Collect x and y 
        item_y = s.values
        item_x = s.keys()

        #Making trace
        trace = go.Scatter(x=item_x,
                  y=item_y,
                  name=title)
        #Append trace to dataset
        data.append(trace)
        
    plot(data,filename='scatter.html')

In [22]:
#make scatter plot for the top20 foods for each cuisine in the world
draw_scatter(top_international_df)

In [24]:
#make scatter plots for different cuisines in each country
#Scatter plot
def draw_scatterFiles(df, file_name):
    data = []
    for item in df.iteritems(): 
        #Extract the title: 
        title = item[0]
        #Extract the series of that title 
        s = item[1]
        #Collect x and y 
        item_y = s.values
        item_x = s.keys()

        #Making trace
        trace = go.Scatter(x=item_x,
                  y=item_y,
                  name=title)
        #Append trace to dataset
        data.append(trace)
        
    plot(data,filename= file_name + '.html')

for country, code in country_and_codes:
    print 'country', country
    df = food_in_country_over_time (top20, final_countries_list, '2004-04-22', '2018-04-22', code)
    draw_scatterFiles(df, country)

country USA
USA
China
France
Germany
UK
India
Italy
Thailand
Spain
Mexico
Ethiopia
Ecuador
Korea
Vietnam
South Africa
country China
USA
China
France
Germany
UK
India
Italy
Thailand
Spain
Mexico
Ethiopia
Ecuador
Korea
Vietnam
South Africa


ValueError: labels ['isPartial'] not contained in axis

In [26]:
#make scatter plots for different cuisines in the world
world_df = food_in_country_over_time (top20, final_countries_list, '2004-04-22', '2018-04-22', '')
draw_scatterFiles(world_df, 'world')

USA
China
France
Germany
UK
India
Italy
Thailand
Spain
Mexico
Ethiopia
Ecuador
Korea
Vietnam
South Africa


<b>Graphing functions follow below</b>

In [27]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from pytrends.request import TrendReq
import pandas as pd 

import plotly.graph_objs as go

init_notebook_mode(connected=True)

import plotly
from plotly.graph_objs import Scatter, Layout
import plotly.graph_objs
import plotly.plotly as py
import plotly.figure_factory as ff

In [28]:
# kw_list = ["Orange","Apple","Berry","Melon","Banana","Pumpkin","Grapes","Dragon Fruit","Watermelon","Blueberry"]

#Function take in a list of keywords, starting date, ending date in forms of 'YYYY-MM-DD YYYY-MM-DD', and region
#Yielding: a normalized Dataframe of keywords as y-axis, 
#timeline break down by months, and values as trend index over 100
#this should only be used for small queries with max 5 items in the keyword (kw) list
def trend_over_time(kw,start_date,end_date,region):
        
    first_five = kw[0:5]
#     second_five = kw[5:]
    
    pytrends = TrendReq(hl='en-US', tz=360)

    #First five 
    pytrends.build_payload(first_five, cat=0, timeframe=start_date + ' ' + end_date, geo=region, gprop='')
    df_1 = pytrends.interest_over_time().drop('isPartial',1)
#     time.sleep(1)
#     #Second dive 
#     pytrends.build_payload(second_five, cat=0, timeframe=start_date + ' ' + end_date, geo=region, gprop='')
#     df_2 = pytrends.interest_over_time().drop('isPartial',1)
    
#     Big_DataFrame = pd.concat([df_1, df_2], axis=1)
    
    return df_1

# result_time = trend_over_time(kw_list,'2016-01-01','2017-12-01','US')

In [29]:
#Scatter plot
def draw_scatter(df,x,y,t):
    data = []
    for item in df.iteritems(): 
        #Extract the title: 
        title = item[0]
        #Extract the series of that title 
        s = item[1]
        #Collect x and y 
        item_y = s.values
        item_x = s.keys()

        #Making trace
        trace = go.Scatter(x=item_x,
                  y=item_y,
                  name=title)
        #Append trace to dataset
        data.append(trace)
    
    layout = dict(title = t, xaxis = dict(title='Time'),
        yaxis = dict(title='Search Index'))
    fig = dict( data=data, layout=layout )
    plot(fig,filename='scatter.html')

In [30]:
#Table
def nine_bins_gen(matrix):
    new_matrix = []
    for row in matrix: 
        new_row = []
        for element in row: 
            if element <= 10: 
                new_row.append(0)
            elif element <= 21:
                new_row.append(1)
            elif element <= 32: 
                new_row.append(2)
            elif element <= 43: 
                new_row.append(3)
            elif element <= 54: 
                new_row.append(4)
            elif element <= 65: 
                new_row.append(5)
            elif element <= 76: 
                new_row.append(6)
            elif element <= 87: 
                new_row.append(7)
            else: 
                new_row.append(8)
        new_matrix.append(new_row)
    return new_matrix

#sudo pip install colorlover
import colorlover as cl
from collections import Counter
import numpy as np

def draw_table(df):
    #Create shades of color 
    val = []
    for l in df.values: 
        for i in l: 
            val.append(i)
    num_shades = str(len(Counter(val)))
    colors = cl.scales['9']['seq']['Blues']
    
    #Create list of columns 
    list_col = []
    for col in df.keys():
        #Add each vertical list to the list
        list_col.append(df[col].values)
    
    #normalize the color scheme
    new_matrix = nine_bins_gen(list_col)
        
    trace = go.Table(type = 'table',
                     header = dict(values = list(df.keys()),
                                   line = dict(color = 'black'),
                                   fill = dict(color = 'black'),
                                   align = 'center',
                                   font = dict(color = 'white', size = 12)),
                     cells = dict(values = list_col,
                                  line = dict(color = [np.array(colors)[element] for element in new_matrix]),
                                  fill = dict(color = [np.array(colors)[element] for element in new_matrix]),
                                  align = 'center',
                                  font = dict(color = 'black', size = 12)))
    data = [trace]
    plot(data, filename='table1.html')

In [31]:
#One kw and region at the moment
def trend_over_region(kw,start_date,end_date,region):
  
    pytrends = TrendReq(hl='en-US', tz=360)
    
    pytrends.build_payload(kw, cat=0, timeframe=start_date + ' ' + end_date, geo=region, gprop='')
    Big_DataFrame = pytrends.interest_by_region()
    
    return Big_DataFrame

result_region = trend_over_region(['Pizza'],'2017-01-01','2017-12-01', 'US')
# result_region

In [32]:
#Source: https://gist.github.com/rogerallen/1583593
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

In [33]:
def draw_US_map(df,title):
    for col in df.columns:
        df[col] = df[col].astype(str)

    scl = [[0.0, 'rgb(247,252,240)'],[0.1, 'rgb(224,243,219)'],[0.2, 'rgb(204,235,197)'],
            [0.4, 'rgb(168,221,181)'],[0.5, 'rgb(123,204,196)'],[0.6, 'rgb(78,179,211)']
          ,[0.7, 'rgb(43,140,190)'],[0.8, 'rgb(8,104,172)'],[1.0, 'rgb(8,64,129)']]
    
    #Extract states and values
    states = pd.Series(df.ix[:,0].keys())
    val = pd.Series(df.ix[:,0].values)
    df['text'] = states + '<br>' +  'Search index: ' + val
    
    #Extract states as abbreviated list
    locations_code = []
    for s in df.ix[:,0].keys():
        locations_code.append(us_state_abbrev[s])
    
    data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = locations_code,
        z = val.astype(float),
        locationmode = 'USA-states',
        text = df['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Search Index")
        ) ]

    layout = dict(
        title = title,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
    fig = dict( data=data, layout=layout )
    plot( fig, filename='d3-cloropleth-map.html' )


#df containing 2 columns (1: list of states, 2: list of index of ONE kw)
draw_US_map(result_region,'Search index for pizza in 2017')

/Users/BrookeWerner/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated



In [34]:
import pandas as pd

#get the Google trends over time for items in dataframe df
def info_by_state (df, start_date, end_date, region, first_item, country=None):
    
    time = start_date + ' ' + end_date
    
    #if a country is specified, only get items for that country
#     if country == None:
    kwlist = df[0].tolist()
#     else:
#         country_items = df.loc[df[1] == country]
#         kwlist = country_items[0].tolist()
        
    if first_item == 'None':
        first_item = kwlist[0]
 
    first = [first_item]
        
    #get info for first 5 items
    first.extend(kwlist[0:4])
        
    pytrends.build_payload(first, cat=0, timeframe= time, geo=region, gprop='')
        
    tempdf = pytrends.interest_by_region(resolution='REGION')
    tempdf.loc['mean'] = tempdf.mean() #add mean as last row
    
    ref_mean = tempdf.loc['mean'][0] #mean of first term, used as reference    
    df = tempdf #final dataframe
    
    #print 'ref1:',ref_mean
    #print first[0]
    
    # start and end indices of items
    start = 0
    end = 4
    
    done = False
    while not done:
        
        start = start + 4
        if (len(kwlist) - 1) > (end + 4):
            end = end + 4

        else:
            end = len(kwlist)
            done = True
        
        #get the info for these items
        items = kwlist[start:end] 
        
        items.append(first_item)
                
        pytrends.build_payload(items, cat=0, timeframe= time, geo='US', gprop='')
        tempdf = pytrends.interest_by_region(resolution='REGION')

        tempdf.loc['mean'] = tempdf.mean() #add mean as last row
        
        #normalize the values
        #equation = x * ref_mean/ ref_mean2
        ref_mean2 = tempdf.loc['mean'][first_item]
        
        #print 'ref2:',ref_mean2
        
        normal_factor = float (ref_mean)/ref_mean2
        tempdf = tempdf.apply(lambda x: x*normal_factor)
        tempdf = tempdf.drop(first_item,1)
        
        #append to the final dataframe
        df = pd.concat([df, tempdf], axis=1)
#     df = df.loc[states_list, :]

    return df

In [35]:
fastfoods = ["Burrito","Doughnut","French fries","Fried chicken", "Hot Dog","Hamburger","Taco", "Ice Cream"]

ff = pd.DataFrame(fastfoods)

allstateinfo = info_by_state (ff, '2017-01-01', '2017-12-31', 'US', 'Hamburger')

In [36]:
dfOfStates = pd.DataFrame((allstateinfo.index[0:51]).tolist()) #get a list of just all the states

In [37]:
allstateinfo['Max'] = allstateinfo.idxmax(axis=1) #

In [38]:
allstateinfo = allstateinfo.drop(allstateinfo.columns[0:8], axis=1)

In [39]:
def indexize(row):
    if (row['Max']) == 'Burrito':
        return 1
    elif (row['Max']) == 'Doughnut':
        return 2
    elif (row['Max']) == 'French fries':
        return 3
    elif (row['Max']) == 'Fried chicken':
        return 4
    elif (row['Max']) == 'Hamburger':
        return 5
    elif (row['Max']) == 'Hot dog':
        return 6
    elif (row['Max']) == 'Chicken Nugget':
        return 7
    elif (row['Max']) == 'Taco':
        return 8
    elif (row['Max']) == 'Pizza':
        return 9
    elif (row['Max']) == 'Ice Cream':
        return 10
    else:
        return 0 #shouldn't ever return this
    
    
allstateinfo['index'] = allstateinfo.apply(lambda row: indexize(row), axis=1) #add a column containing a number that indicates which food is the most popular

In [40]:
allstateinfo = allstateinfo.drop('mean')

In [41]:
allstateinfo

,Max,index
geoName,,
Alabama,Taco,8
Alaska,Taco,8
Arizona,Taco,8
Arkansas,Taco,8
California,Taco,8
Colorado,Taco,8
Connecticut,Ice Cream,10
Delaware,Ice Cream,10
District of Columbia,Taco,8


In [42]:
# same thing as above but including pizza AKA pizza takes over

fastfoods = ["Burrito","Doughnut","French fries","Fried chicken","Hamburger","Hot dog", "Taco", "Pizza"]

ff = pd.DataFrame(fastfoods)

allstateinfoP = info_by_state (ff, '2017-01-01', '2017-12-31', 'US', 'Hamburger')

dfOfStates = pd.DataFrame((allstateinfo.index[0:51]).tolist()) #get a list of just all the states

allstateinfoP['Max'] = allstateinfoP.idxmax(axis=1) #

allstateinfoP = allstateinfoP.drop(allstateinfoP.columns[0:8], axis=1)

def indexize(row):
    if (row['Max']) == 'Burrito':
        return 1
    if (row['Max']) == 'Doughnut':
        return 2
    if (row['Max']) == 'French fries':
        return 3
    if (row['Max']) == 'Fried chicken':
        return 4
    if (row['Max']) == 'Hamburger':
        return 5
    if (row['Max']) == 'Hot dog':
        return 6
    if (row['Max']) == 'Chicken Nugget':
        return 7
    if (row['Max']) == 'Taco':
        return 8
    if (row['Max']) == 'Pizza':
        return 9
    
allstateinfoP['index'] = allstateinfoP.apply(lambda row: indexize(row), axis=1) #add a column containing a number that indicates which food is the most popular

allstateinfoP = allstateinfoP.drop('mean')
allstateinfoP

,Max,index
geoName,,
Alabama,Pizza,9
Alaska,Pizza,9
Arizona,Pizza,9
Arkansas,Pizza,9
California,Pizza,9
Colorado,Pizza,9
Connecticut,Pizza,9
Delaware,Pizza,9
District of Columbia,Pizza,9


In [43]:
#draws US map for different food in each state
def draw_US_map2(df,title):
    for col in df.columns:
        df[col] = df[col].astype(str)

    scl = [[0.0, 'rgb(247,252,240)'],[0.1, 'rgb(224,243,219)'],[0.2, 'rgb(204,235,197)'],
            [0.4, 'rgb(168,221,181)'],[0.5, 'rgb(123,204,196)'],[0.6, 'rgb(78,179,211)']
          ,[0.7, 'rgb(43,140,190)'],[0.8, 'rgb(8,104,172)'],[1.0, 'rgb(8,64,129)']]
    
    #Extract states and values
    states = pd.Series(df.ix[:,0].keys())
    val = pd.Series(df.ix[:,1].values)
    word = pd.Series(df.ix[:,0].values)
    
    df['text'] = states + '<br>' +  'Popular: ' + word
   
    #Extract states as abbreviated list
    locations_code = []
    for s in df.ix[:,0].keys():
        locations_code.append(us_state_abbrev[s])
    
    data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = locations_code,
        z = val.astype(float),
        locationmode = 'USA-states',
        text = word,
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
#         colorbar = dict(
#             title = "Search Index"),
        showscale = False
    )]

    layout = dict(
        title = title,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
    fig = dict( data=data, layout=layout )
    plot( fig, filename='d3-cloropleth-map.html' )

In [44]:
draw_US_map2(allstateinfo, "Most Popular Fast Foods in Each State (2017)")

/Users/BrookeWerner/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated



In [45]:
draw_US_map2(allstateinfoP, "Most Popular Fast Foods in Each State Including Pizza (2017)")

/Users/BrookeWerner/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated



In [51]:
#Import the data from fusion list 
fusion_list = csvListMaker("Fusion.csv")[1:]
fusion_df = pd.DataFrame(fusion_list)
fusion_df = info_over_time(fusion_df,'2010-01-01', '2018-04-31', 'US', 'None', country=None)

#Draw the scatter plot based on the top most searched 20
draw_scatter(get_top20(fusion),"X","Y","Fusion")

ResponseError: The request failed: Google returned a response with code 500.